In [3]:
from azureml.core import Workspace, Dataset, Datastore
import os
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Run

import azureml.core
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ComputeTarget

from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Experiment, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import Pipeline



In [3]:
pip install azureml-pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.7/313.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 122.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.6/249.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: azureml-core
    Found existing installation: azureml-core 1.56.0
    Uninstalling azureml-core-1.56.0:
      Successfully uninstalled azureml-core-1.56.0
Note: you may need to restart the kernel to use updated packages.


In [17]:
ws = Workspace.from_config("Users/mypersonall3099/Online_retail/config.json") 


In [29]:
compute_name = "OnlineRetail"
#vm_size = "Standard_E4ds_v4"
vm_size = "Standard_E8ds_v4"
compute_target = ws.compute_targets[compute_name]

In [55]:
#Declaring environment
aml_config = RunConfiguration()
aml_config.target = compute_target 

USE_CURATEDUENV = False 
if USE_CURATEDUENV:
    curated_env = Environment.get(workspace = ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
    aml_config.environment = curated_env
else:
    aml_config.environment.python.user_managed_dependencies = False
    aml_config.environment.python.conda_dependencies_file = 'environment.yml'

Property conda_dependencies_file is deprecated. Use Environment.from_conda_specification


In [56]:
#Pipeline
read_data = 'data_wrangling.py'
prep = 'preprocessing.py'
model = 'modelling.py'

#Script initialization
py_script_run_read = PythonScriptStep(
                script_name = read_data,
                compute_target = compute_target,
                arguments=['--input-data','online_retail_II.csv'],
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_prep = PythonScriptStep(
                script_name = prep,
                compute_target=compute_target,
                arguments=['--prep','wranggled.csv'],
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_model = PythonScriptStep(
                script_name = model,
                compute_target=compute_target,
                arguments=['--train','preprocessed.csv'],
                runconfig = aml_config,
                allow_reuse=False)

pipeline_step = [py_script_run_read, py_script_run_prep, py_script_run_model]
pipeline_1 = Pipeline(workspace=ws, steps=[pipeline_step])

In [57]:
# Experiment

pipeline_run = Experiment(ws, "First_run_5").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Created step data_wrangling.py [06c99747][82dff221-8388-48ee-bada-aebb224c7aff], (This step will run and generate new outputs)
Created step preprocessing.py [a20c2698][927a62f1-dcfc-46cf-bf54-07fbdb15dc85], (This step will run and generate new outputs)
Created step modelling.py [e1ced509][261a4aef-0784-40bb-843f-274f7d33d9ad], (This step will run and generate new outputs)
Submitted PipelineRun 4d78068b-145d-450d-8b54-9d727cf1a388
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/4d78068b-145d-450d-8b54-9d727cf1a388?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRunId: 4d78068b-145d-450d-8b54-9d727cf1a388
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/4d78068b-145d-450d-8b54-9d727cf1a388?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRu

Ran into a deserialization error. Ignoring since this is failsafe deserialization
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/msrest/serialization.py", line 1509, in failsafe_deserialize
    return self(target_obj, data, content_type=content_type)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/msrest/serialization.py", line 1375, in __call__
    data = self._unpack_content(response_data, content_type)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/msrest/serialization.py", line 1543, in _unpack_content
    raise ValueError("This pipeline didn't have the RawDeserializer policy; can't deserialize")
ValueError: This pipeline didn't have the RawDeserializer policy; can't deserialize
Ran into a deserialization error. Ignoring since this is failsafe deserialization
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/msrest/ser

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. For more details and logs, please go to the job detail page and check the child jobs.",
        "messageFormat": "Pipeline has failed child jobs. {0}",
        "messageParameters": {},
        "referenceCode": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus2",
    "location": "eastus2",
    "time": "2024-08-17T14:23:07.755223Z",
    "componentName": ""
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Pipeline has failed child jobs. For more details and logs, please go to the job detail page and check the child jobs.\",\n        \"messageFormat\": \"Pipeline has failed child jobs. {0}\",\n        \"messageParameters\": {},\n        \"referenceCode\": \"Pipeli

In [ ]:
# Experiment 2

pipeline_run = Experiment(ws, "Second_run").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)